### **Springboard - Unit 8.3 Case Study**
 **PART 2: SQLite | Questions 10 - 13**

The purpose of this notebook is to answer the questions using the the country club data exported from PHPMyAdmin, and connected to a local SQLite instance.

**Imports**

In [1]:
# Importing 

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine
import os

In [2]:
os.getcwd()

'/Users/frankyaraujo/Development/Springboard_Main/GitHubDir/Springboard'

**Local SQLite Instance**

In [3]:
# Create database engine and getting table names

engine = create_engine('sqlite:///Springboard - Unit 8.3 Case Study - Country Club sqlite_db_pythonsqlite.db')
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


/var/folders/l1/kpr_34xn51ndl39jn5r_rqn00000gn/T/ipykernel_50934/619746547.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


In [50]:
# Testing the connection

con = engine.connect()
rs = con.execute("SELECT * FROM Members LEFT JOIN Bookings USING(memid) LEFT JOIN Facilities USING(facid)")
bookings_facilities_members = pd.DataFrame (rs.fetchall())
bookings_facilities_members.columns = rs.keys()
con.close()

bookings_facilities_members.head(2)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate,bookid,facid,starttime,slots,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00,2.0,6.0,2012-07-03 18:00:00,2.0,Squash Court,3.5,17.5,5000.0,80.0
1,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00,9.0,4.0,2012-07-04 15:00:00,2.0,Massage Room 1,9.9,80.0,4000.0,3000.0


### Q10. 
**Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!**

In [55]:
con = engine.connect()
query1="SELECT subquery.Facility, subquery.Revenue \
FROM(SELECT f.name AS Facility, SUM(CASE WHEN b.memid = 0 THEN b.slots*f.guestcost \
                 ELSE b.slots*membercost END) AS Revenue \
FROM Facilities AS f \
LEFT JOIN Bookings AS b \
USING(facid) \
GROUP BY f.name \
ORDER BY revenue DESC) AS subquery \
WHERE subquery.Revenue < 1000;"
rs = con.execute(query1)
q10_answer = pd.DataFrame (rs.fetchall())
q10_answer.columns = rs.keys()
con.close()

print(q10_answer)

        Facility  Revenue
0     Pool Table      270
1  Snooker Table      240
2   Table Tennis      180


### Q11.
**Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order**


In [66]:
con = engine.connect()
query2="""SELECT subquery.Member_name, 
                firstname || ' ' || surname AS Recommended_by
FROM (SELECT 
    m.firstname || ' ' || m.surname AS Member_name, 
    m.recommendedby
FROM Facilities AS f
LEFT JOIN (SELECT*
           FROM Bookings
           WHERE memid<>0) AS b
USING(facid)
LEFT JOIN Members AS m        
USING(memid)
GROUP BY Member_name) AS subquery
LEFT JOIN Members
ON subquery.recommendedby = memid
WHERE memid<>0
ORDER BY subquery.Member_name;"""

rs = con.execute(query2)
q11_answer = pd.DataFrame (rs.fetchall())
q11_answer.columns = rs.keys()
con.close()

print(q11_answer)

                Member_name     Recommended_by
0            Anna Mackenzie       Darren Smith
1                Anne Baker    Ponder Stibbons
2              Charles Owen       Darren Smith
3               David Jones    Janice Joplette
4              David Pinker     Jemima Farrell
5             Douglas Jones        David Jones
6             Erica Crumpet        Tracy Smith
7            Florence Bader    Ponder Stibbons
8            Gerald Butters       Darren Smith
9          Henrietta Rumney    Matthew Genting
10  Henry Worthington-Smyth        Tracy Smith
11               Jack Smith       Darren Smith
12          Janice Joplette       Darren Smith
13              Joan Coplin      Timothy Baker
14                John Hunt  Millicent Purview
15          Matthew Genting     Gerald Butters
16        Millicent Purview        Tracy Smith
17               Nancy Dare    Janice Joplette
18          Ponder Stibbons       Burton Tracy
19         Ramnaresh Sarwin     Florence Bader
20           

### Q12.
**Q12: Find the facilities with their usage by member, but not guests**

In [67]:
con = engine.connect()
query3= """SELECT firstname || ' ' || surname AS Member_name,
    f.name AS Facility, SUM(b.slots) AS Slots_booked
FROM Facilities AS f
LEFT JOIN (SELECT*
           FROM Bookings
           WHERE memid<>0) AS b
USING(facid) 
LEFT JOIN Members AS m 
USING(memid)
GROUP BY Member_name;"""

rs = con.execute(query3)
q12_answer = pd.DataFrame (rs.fetchall())
q12_answer.columns = rs.keys()
con.close()

print(q12_answer)

                Member_name         Facility  Slots_booked
0            Anna Mackenzie  Badminton Court           231
1                Anne Baker   Tennis Court 1           296
2              Burton Tracy   Tennis Court 1           366
3              Charles Owen   Tennis Court 1           345
4              Darren Smith   Tennis Court 2           685
5             David Farrell   Tennis Court 1            50
6               David Jones   Tennis Court 1           305
7              David Pinker   Tennis Court 1           159
8             Douglas Jones   Tennis Court 1            37
9             Erica Crumpet   Tennis Court 1            17
10           Florence Bader   Tennis Court 1           237
11           Gerald Butters   Tennis Court 1           409
12         Henrietta Rumney   Tennis Court 2            38
13  Henry Worthington-Smyth  Badminton Court            60
14      Hyacinth Tupperware  Badminton Court            28
15               Jack Smith   Tennis Court 1           2

### Q13.
**Q13: Find the facilities usage by month, but not guests**

In [69]:
con = engine.connect()
query4= """SELECT 
    f.name,
    strftime('%m',b.starttime) AS month,
    SUM(b.slots) AS Slots_booked    
FROM Facilities AS f
LEFT JOIN (SELECT*
           FROM Bookings
           WHERE memid<>0) AS b
USING(facid) 
LEFT JOIN Members AS m 
USING(memid)
GROUP BY f.name,month;"""

rs = con.execute(query4)
q13_answer = pd.DataFrame (rs.fetchall())
q13_answer.columns = rs.keys()
con.close()

print(q13_answer)

               name month  Slots_booked
0   Badminton Court    07           165
1   Badminton Court    08           414
2   Badminton Court    09           507
3    Massage Room 1    07           166
4    Massage Room 1    08           316
5    Massage Room 1    09           402
6    Massage Room 2    07             8
7    Massage Room 2    08            18
8    Massage Room 2    09            28
9        Pool Table    07           110
10       Pool Table    08           303
11       Pool Table    09           443
12    Snooker Table    07           140
13    Snooker Table    08           316
14    Snooker Table    09           404
15     Squash Court    07            50
16     Squash Court    08           184
17     Squash Court    09           184
18     Table Tennis    07            98
19     Table Tennis    08           296
20     Table Tennis    09           400
21   Tennis Court 1    07           201
22   Tennis Court 1    08           339
23   Tennis Court 1    09           417
